In [1]:
import os
os.environ["XUVTOP"] = "/usr/local/ssw/packages/chianti/dbase/"
from scipy.io import readsav
import xarray as xr
import numpy as np
from dateutil.tz import gettz
from datetime import datetime
from datetime import date
import dask, pdb
from pathlib import Path

import matplotlib.pyplot as plt
from matplotlib import colors

from muse.synthesis.synthesis import vdem_synthesis, vdem_syn_join_slits_wvl, ph2e, exposure_time
from muse.instr.psf import  convolve_model_psf, oversampled_wavelength_to_sg
from muse.instr.gausslobes import gausslobes
from muse.instr.detect_sidelobes import remove_yx_sidelobes
from muse.instr.utils import create_eff_area_xarray, ci2sg_response_format
from muse.transforms.transforms import muse_fov, reshape_x2rs, reshape_rs2x, muse_interp_xyw2rssg

from muse.utils.utils import  deweight_func, sub_contaminants, read_response, weight_func, sub_positive
from muse.instr.utils import sum_lines_per_band, sum_lines_slits_per_band
from muse.inversions.inversions import vdem_solve

from muse.utils.fitting import masker
from muse.variables import CENTROID_UNCERT_PROMISED, EXP_NOM
from muse.utils.fitting import gaussian_fitting_analysis,calculate_moments_gaussian
from muse.utils.fitting import main_gaussian_fitter,_vet_new_local_max,fit_gaussian
from muse.utils.fitting import fit_gaussian_remove_badspec,local_max
from muse.utils.miscellaneous import plot_gfat2c, generate_outliers_newer, get_outliers_newer, plot_with_chisquare

from sklearn.linear_model import Ridge as Ridge
from sklearn.linear_model import LassoLars as LassoLars
author="JMS"
WEST = gettz("US/Western")

In [2]:
data_path = '/home/ec2-user/SageMaker/data/565000/'
response= xr.open_zarr(data_path + 'response_565000.zarr').compute()
response["line"] = ['Fe XIX', 'Fe XXI', '108 missing main line', 'Fe IX',
                    '171 missing main line', 'Fe XV', '284 missing main line']
# reduce the volume of response function
response = response.sel(logT=np.arange(5.0, 7.1, 0.1), 
                        vdop = np.arange(-780, 1020, 20), 
                        method='nearest')
respsum = sum_lines_per_band(response)

2025-02-28 15:18:30.926 | INFO     | muse.instr.utils:sum_lines_per_band:1283 - Swapping lines with band was required


In [3]:
if False:
    spec_gt = xr.open_zarr(data_path+'spec_m-flare-muram_los2_trac_565000_core.zarr').compute()
    spec_gt["flux"].attrs = dict(units="ph / s")
    spec_gt["gain"] = xr.DataArray([10]*3, dims=['band'])
    spec_obs = sum_lines_slits_per_band(spec_gt)
    spec_obs.to_zarr(data_path+'spec_obs.zarr', mode='w')
    
    spec_gt_70lpi = xr.open_zarr(data_path+'spec_m-flare-muram_los2_trac_565000_70LPI.zarr').compute()
    spec_gt_70lpi["flux"].attrs = dict(units="ph / s")
    spec_gt_70lpi["gain"] = xr.DataArray([10]*3, dims=['band'])
    spec_obs_70lpi = sum_lines_slits_per_band(spec_gt_70lpi)
    spec_obs_70lpi.to_zarr(data_path+'spec_obs_70lpi.zarr', mode='w')
else:
    # spec_obs = xr.open_zarr(data_path+'spec_obs.zarr').compute()
    spec_obs_70lpi = xr.open_zarr(data_path+'spec_obs_70lpi.zarr').compute()

In [4]:
spec_obs_70lpi.HISTORY

["from_hel2vdem(snap=565000, dopaxis=dopaxis, lgtaxis=lgtaxis, logdensaxis=None, tg=[], vel=[], dens=[], ems=[], axis=2, interp_vdem=False, radyn=False, highrun=False, xyrotation=False, opacity=None, trac=True, trac_coeff=3, kwargs={}), muse_fov(vdem=vdem, dx_pix=0.4, dy_pix=0.167, nslits=35, nraster=11, restype=match_res_tile, mode=wrap, sub_interpolation=2), reshape_x2rs(ds=ds, nslits=35, nraster=11), reshape_rs2x(ds=ds, nslits=35, nraster=11), vdem_syn_join_slits_wvl(response=response, vdem=vdem, islits=(2, 32), sum_over=('logT', 'vdop'), band=(108, 171, 284), line=('Fe XIX', 'Fe IX', 'Fe XV'), use_gpu=True), convolve_model_psf(spectrum=spectrum, model=<function gausslobes at 0x7f864bbc8860>, fixed_psf_per_channel=False, band=[284], inplace=False, kwargs={'oversample_x': 3, 'slit_dim': 404, 'lpi': 70}), reshape_x2rs(ds=ds, nslits=35, nraster=11), muse_interp_xyw2rssg(spec=spec, response=response, band=band, line=('Fe XIX', 'Fe IX', 'Fe XV'), method=cubic, kwargs={}), sum_lines_slits

In [5]:
aa = gausslobes()

In [7]:
np.sum(aa)

0.7926032302790151

In [4]:
def get_scaling(spec):
    sz = [spec_obs_70lpi[i].size for i in spec_obs_70lpi.flux.dims]
    scaling = xr.DataArray(np.ones(sz), 
                           dims=spec.flux.dims,
                           coords=spec.flux.coords,
                           attrs=dict(description="Intensity(x, y, line_wvl)",
                                      units="DN"))
    scaling *= np.sqrt(spec.flux + spec.flux.max()/1e4)/1e2
    return scaling

In [5]:
norm = colors.PowerNorm(0.3, 0, 1e4)
npix = 2
velocity_range = 400
line = ['Fe XV']

In [ ]:
%%capture output
# 1st
# Disambiguation without any trick on the diffraction. 

respsum['SG_resp'] = respsum['SG_resp']*0.81**2
num = ['1st', '2nd', '3rd', '4th', '5th']
spec0 = spec_obs_70lpi.copy(deep=True)
for i in range(5):
    print(num[i])
    if i != 0 :
        vdem_inv = vdem_solve(spec0,
                              respsum,
                              model = LassoLars(alpha=1.e-2, fit_intercept=False, positive=True, eps=1e-40),
                              scaling=get_scaling(spec0))
        vdem_inv.to_zarr(data_path+f'inv_vdem_{num[i]}.zarr', mode='w')
    else: 
        vdem_inv = xr.open_zarr(data_path+f'inv_vdem_{num[i]}.zarr').compute()
    spec_inv = vdem_syn_join_slits_wvl(response, vdem_inv)
    spec_inv['gain'] = xr.DataArray([10]*3, dims='band')
    diff_pat = convolve_model_psf(spec_inv, model=gausslobes,**{'oversample_x': 3, 'slit_dim': 404}, 
                                  band=[284], no_core=True)
    diff_pat['gain'] = xr.DataArray([10]*3, dims='band')
    diff_pat, resp = muse_interp_xyw2rssg(diff_pat, response) 
    diff_pat['gain'] = xr.DataArray([10]*3, dims='band')
    diff_pat = sum_lines_slits_per_band(diff_pat)
    diff_pat['gain'] = xr.DataArray([10]*3, dims='band')
    spec_sub = sub_positive(spec0, diff_pat)
    spec_sub.to_zarr(data_path+f'spec_sub_{num[i]}.zarr', mode='w')
    mom_sub = gaussian_fitting_analysis(spec_sub, response, line_band_key=line,
                                       kwargs_obs={"npix": npix, "velocity_range": velocity_range})
    mom_sub.to_zarr(data_path+f'mom_sub_{num[i]}.zarr', mode='w')
    spec0 = spec_sub.copy()
    

In [ ]:
output.show()

In [8]:
print('a')

a


In [21]:
import time
tt = 0
while True:  
    print (f'{tt} min', end="\r")
    tt += 1
    time.sleep(60)

65 min

KeyboardInterrupt: 

In [ ]:
func_list = [
    # preparing response function  (summing lines)
    "respsum & sum_lines_per_band(resp)",
    "respsum.SG_resp & respsum.SG_resp*0.81**2",
    # preparing for disambiguation (some normalization between bands)
    "spec, respsum & weight_func(spec, respsum, fact = {0: 0.10, 1: 5., 2: 2.})", 
    # disambiguation
    "sol & vdem_solve(spec, respsum, model = LassoLars(alpha=1.e-3, fit_intercept=True, normalize=True, eps=np.finfo(float).eps))",
    # remove the normalization between bands
    "spec, respsum, sol & deweight_func(spec, respsum, sol, fact = {0: 0.10, 1: 5., 2: 2.})",
]
if True:
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore')
        spec_psf_a, resp_a, sol_psf_a = adrian_apprach(specsum_psf, response, func_list)
    spec_psf_a.to_zarr("spec_psf_a.zarr", mode='w')
    resp_a.to_zarr("resp_a.zarr", mode='w')
    sol_psf_a.to_zarr("sol_psf_a.zarr", mode='w')
else:
    spec_psf_a = xr.open_zarr("spec_psf_a.zarr", consolidated=True)
    sol_psf_a = xr.open_zarr("sol_psf_a.zarr", consolidated=True)

NameError: name 'specsum_psf' is not defined

In [ ]:
# Compute some "fake" diffraction pattern and substract from the original data. 
func_list = [
    # synthesize as a function of wvl (no spectrograph)
    "spec & vdem_syn_join_slits_wvl(resp, sol)",
    # obtain the diffraction pattern in the spectrum (I*PSF/(90%^2) - I), i.e., diffraction pattern in spectrum, 
    "diff_pat & convolve_model_psf(spec, model=gausslobes,**{'oversample_x': 3, 'slit_dim': 404}, band=[284], no_core=True)", # only spikes 
    # MUSE stuff on the diffraction pattern, i.e., slits, raster, etc. 
    "diff_pat, resp & muse_interp_xyw2rssg(diff_pat, resp)", 
    # MUSE Sum slits and contaminants for the diffraction pattern
    "diff_pat & sum_lines_slits_per_band(diff_pat)",
    # Adrians trick: Spec(orig) - diffraction pattern
    "spec & sub_possitive(spec_orig, diff_pat)", 
]
if False:
    spec2_psf_a, resp2_a, sol2_psf_a = adrian2_apprach(spec_psf_a, response, sol_psf_a, specsum_psf, func_list)
    spec2_psf_a.to_zarr("spec2_psf_a.zarr", mode='w')
    resp2_a.to_zarr("resp2_a.zarr", mode='w')
    sol2_psf_a.to_zarr("sol2_psf_a.zarr", mode='w')
else:
    spec2_psf_a = xr.open_zarr("spec2_psf_a.zarr", consolidated=True)
    sol2_psf_a = xr.open_zarr("sol2_psf_a.zarr", consolidated=True)


In [ ]:
specsum_psf.flux.isel(step=0,band=2).plot.imshow(norm=colors.PowerNorm(0.3), cmap='jet', figsize=(12,6),vmin=0,vmax=1e4)

In [ ]:
spec_psf_a.flux.isel(step=0,band=2).plot.imshow(norm=colors.PowerNorm(0.3), cmap='jet', figsize=(12,6),vmin=0,vmax=1e4)

In [ ]:
specsum_psf = sum_lines_slits_per_band(muse_spect_psf)

In [ ]:
# 2nd
# Disambiguation without any trick on the diffraction. 
func_list = [
    # preparing response function  (summing lines)
    "respsum & sum_lines_per_band(resp)",
    "respsum.SG_resp & respsum.SG_resp*0.81**2",
    # preparing for disambiguation (some normalization between bands)
    "spec, respsum & weight_func(spec, respsum, fact = {0: 0.10, 1: 5., 2: 2.})", 
    # disambiguation
    "sol & vdem_solve(spec, respsum, model = LassoLars(alpha=1.e-3, fit_intercept=True, normalize=True, eps=np.finfo(float).eps, random_state=42, jitter=1),mask=mask)",
    # remove the normalization between bands
    "spec, respsum, sol & deweight_func(spec, respsum, sol, fact = {0: 0.10, 1: 5., 2: 2.})",
]
if True:
    spec3_psf_a, resp3_a, sol3_psf_a = adrian_apprach(spec2_psf_a, response, func_list,mask=mask)
    spec3_psf_a.to_zarr("spec3_psf_a.zarr", mode='w')
    resp3_a.to_zarr("resp3_a.zarr", mode='w')
    sol3_psf_a.to_zarr("sol3_psf_a.zarr", mode='w')
else:
    spec3_psf_a = xr.open_zarr("spec3_psf_a.zarr", consolidated=True)
    sol3_psf_a = xr.open_zarr("sol3_psf_a.zarr", consolidated=True)

In [ ]:
specsum_psf = sum_lines_slits_per_band(muse_spect_psf)

In [ ]:
# Compute some "fake" diffraction pattern and substract from the original data. 
func_list = [
    # synthesize as a function of wvl (no spectrograph)
    "spec & vdem_syn_join_slits_wvl(resp, sol)",    
    # obtain the diffraction pattern in the spectrum (I*PSF/(90%^2) - I), i.e., diffraction pattern in spectrum, 
    "diff_pat & convolve_model_psf(spec, model=gausslobes,**{'oversample_x': 3, 'slit_dim': 404}, band=[284], no_core=True)", # only spikes 
    # MUSE stuff on the diffraction pattern, i.e., slits, raster, etc. 
    #diff_at *= 1.62 = 1/T**2
    "diff_pat, resp & muse_interp_xyw2rssg(diff_pat, resp)", 
    # MUSE Sum slits and contaminants for the diffraction pattern
    "diff_pat & sum_lines_slits_per_band(diff_pat)",
    # Adrians trick: Spec(orig) - diffraction pattern
    "spec & sub_possitive(spec_orig, diff_pat)", 
]
if True:
    spec4_psf_a, resp4_a, sol4_psf_a = adrian2_apprach(spec3_psf_a, response, sol3_psf_a, specsum_psf, func_list)
    spec4_psf_a.to_zarr("spec4_psf_a.zarr", mode='w')
    resp4_a.to_zarr("resp4_a.zarr", mode='w')
    sol4_psf_a.to_zarr("sol4_psf_a.zarr", mode='w')
else:
    spec4_psf_a = xr.open_zarr("spec4_psf_a.zarr", consolidated=True)
    sol4_psf_a = xr.open_zarr("sol4_psf_a.zarr", consolidated=True)


In [ ]:
# 3rd
# Disambiguation without any trick on the diffraction. 
func_list = [
    # preparing response function  (summing lines)
    "respsum & sum_lines_per_band(resp)",
    "respsum.SG_resp & respsum.SG_resp*0.81**2",
    # preparing for disambiguation (some normalization between bands)
    "spec, respsum & weight_func(spec, respsum, fact = {0: 0.10, 1: 5., 2: 2.})", 
    # disambiguation
    "sol & vdem_solve(spec, respsum, model = LassoLars(alpha=1.e-3, fit_intercept=True, normalize=True, eps=np.finfo(float).eps, random_state=42),mask=mask)",
    # remove the normalization between bands
    "spec, respsum, sol & deweight_func(spec, respsum, sol, fact = {0: 0.10, 1: 5., 2: 2.})",
]
if True:
    spec5_psf_a, resp5_a, sol5_psf_a = adrian_apprach(spec4_psf_a, response, func_list,mask=mask)
    spec5_psf_a.to_zarr("spec5_psf_a.zarr", mode='w')
    resp5_a.to_zarr("resp5_a.zarr", mode='w')
    sol5_psf_a.to_zarr("sol5_psf_a.zarr", mode='w')
else:
    spec5_psf_a = xr.open_zarr("spec5_psf_a.zarr", consolidated=True)
    sol5_psf_a = xr.open_zarr("sol5_psf_a.zarr", consolidated=True)

In [ ]:
specsum_psf = sum_lines_slits_per_band(muse_spect_psf)